In [39]:
import pandas as pd
import requests
import json
import base64
import tensorflow as tf

In [51]:
def load_data(file_path: str) -> pd.DataFrame:
    """Reads the CSV dataset and returns it as a DataFrame."""
    return pd.read_csv(file_path)


def extract_input_features(data: pd.DataFrame, row_index: int) -> dict:
    """Extracts the input features from a specific row in the DataFrame."""
    return data.iloc[row_index, :8].to_dict()


def convert_to_tf_example(inputs: dict) -> str:
    """Converts the input dictionary into a tf.train.Example format for model consumption."""
    feature_spec = {}

    for key, value in inputs.items():
        if isinstance(value, float):
            feature_spec[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        elif isinstance(value, int):
            feature_spec[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, str):
            feature_spec[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

    example = tf.train.Example(features=tf.train.Features(feature=feature_spec)).SerializeToString()
    return base64.b64encode(example).decode()


def prepare_prediction_request(inputs: dict) -> str:
    """Prepares the JSON request data to be sent to the prediction endpoint."""
    example_b64 = convert_to_tf_example(inputs)
    request_payload = {
        "signature_name": "serving_default",
        "instances": [{"examples": {"b64": example_b64}}]
    }
    return json.dumps(request_payload)


def get_prediction(endpoint: str, json_data: str) -> str:
    """Sends a request to the endpoint and returns the prediction result."""
    response = requests.post(endpoint, data=json_data)
    
    if response.status_code != 200:
        return "Error: Request failed with status code {}".format(response.status_code)
    
    prediction = response.json().get("predictions")
    if prediction:
        prediction_value = prediction[0][0]
        return "Diabetes" if prediction_value >= 0.5 else "No diabetes"
    return "Error: No predictions found in response."


def main():
    data_file = 'data/diabetes_prediction_dataset.csv'
    row_index = 53
    endpoint = "https://diabetes-ops-production.up.railway.app/v1/models/diabetes-classification-model:predict"

    data = load_data(data_file)
    inputs = extract_input_features(data, row_index)
    json_data = prepare_prediction_request(inputs)
    result = get_prediction(endpoint, json_data)
    print('Result:', result)


if __name__ == "__main__":
    main()


Result: Diabetes
